In [1]:
import requests, pprint
from bs4 import BeautifulSoup
from urllib import parse
from datetime import datetime

# JS로 실행되는 Paging을 Crawling하기 위해서는 selenium Library를 이용하여 직접 화면의 JS를 실행해야한다.
# ex) 
# import time
# from selenium import webdriver
# from itertools import count
# from bs4 import BeautifulSoup
# import pandas as pd
# 
# def crawling_goobne():
#     url = 'http://www.goobne.co.kr/store/search_store.jsp'
# 
#     # \U는 유니코드로 인식되기 때문에 \\U와 같이 escape 처리했다.
#     wd = webdriver.Chrome('C:\\Users\samsung\python\webdriver\chromedriver.exe')
#     wd.get(url)
# 
#     RESULT_DIRECTORY = '__results__/crawling'
#     results = []
#     for page in count(1):
#         script = 'store.getList(%d)' % page  # 굽네치킨에서 사용하는 페이지를 이동시키는 js 코드
#          wd.execute_script(script)  # js 실행
#          time.sleep(5)              # 크롤링 로직을 수행하기 위해 5초정도 쉬어준다.
# 
#         html = wd.page_source
#         bs = BeautifulSoup(html, 'html.parser')
#         tag_body = bs.find('tbody', attrs={'id': 'store_list'})
#         tags_tr = tag_body.findAll('tr')
#         # print(tags_tr)
# 
#         if tags_tr[0].get('class') is None:  # 맨 마지막 페이지인 102에서는 class='on lows'가 없다. => 종료 조건
#             break
# 
#         for tag_tr in tags_tr:
#             strings = list(tag_tr.strings)
#             name = strings[1]
#             address = strings[6]
#             results.append((name, address))
# 
#     table = pd.DataFrame(results, columns=['name', 'address'])
#     table.to_csv('{0}/table_goobne.csv'.format(RESULT_DIRECTORY), encoding='utf-8', mode='w')

def get_html(_url, _id):
    r = requests.get(_url + _id)
    html = BeautifulSoup(r.content, 'html.parser')
    """
    Function select is a copy selector reading from tag. 
        div { id: section_review, class: detail_cont detail_customer_review }
            div { id: _review_filter_container }
                ul { id: _review_list, class: lst_review }
                    li { class: thumb_nail } or li { class: thumb_nail_open } javascript function execution !
    BeatifulSoup API
        select(), contents(), extract()
    """
    li_size = len(html.find_all("li", attrs={'class': 'thumb_nail'}))
    crawling_arr_set = []
    while li_size:
        average_star = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > a > span.curr_avg > strong" % li_size)[0].text
        store = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(1)" % li_size)[0].text
        user_name = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(2)" % li_size)[0].text
        write_date = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(3)" % li_size)[0].text
        title_text = html.select("#_review_list > li:nth-child(%d) > div > p > strong" % li_size)[0].text
        body_text = html.select("#_review_list > li:nth-child(%d) > div > div.atc" % li_size)[0].text
        # print(li_size)
        # print(average_star, store, user_name, write_date, title_text, body_text)
        crawling_arr_set.append({
            "aver_star": average_star,
            "user": user_name,
            "store": store,
            "date": write_date,
            "title": title_text,
            "body": body_text,
            "product_id": _id
        })
        li_size -= 1
    return crawling_arr_set

'''
    Chrome Naver Shopping Raw URL
    https://search.shopping.naver.com/detail/detail.nhn?nvMid=21076849694&NaPm=ct%3Dkad8qf08%7Cci%3D0yW0002VbGbs1XUxwvkS%7Ctr%3Dpla%7Chk%3D6066b0a1499f4f77890d54202d9808691c54a75d"
    https://search.shopping.naver.com/detail/detail.nhn?nvMid=13810664630&NaPm=ct%3Dkbkecz08%7Cci%3Da630b02bff9323064460419c72916238067e54e6%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D6a4ad9a61c99a492bd2beb295b5e65864a233a92"
    parse.unquote() is Decoding the URL parameter.
'''

NAVER_SHOPPING_BASE_URL = "https://search.shopping.naver.com/detail/detail.nhn?nvMid="
KUKU_AC_25W20FPMO = "21076849694"
DYSON_PURE_COOL_TP_04 = "13810664630"
ITC_PURIWAY_PW_150 = "20850829749"
LG_PURICARE_AS300DWFA = "21285691276"
WINIX_FRIME_APRM833_JWK = "21393999339"
SAMSUNG_BLUESKY_AX34N3020WWD = "13611653308"
SKMAGIC_ACL_120Z0 = "17586234530"
ELECTROMAN_DAP_2215NAWHEM = "22163383505"

product_arr = [KUKU_AC_25W20FPMO,DYSON_PURE_COOL_TP_04,ITC_PURIWAY_PW_150,LG_PURICARE_AS300DWFA,
               WINIX_FRIME_APRM833_JWK, SAMSUNG_BLUESKY_AX34N3020WWD, SKMAGIC_ACL_120Z0]
print(product_arr)

['21076849694', '13810664630', '20850829749', '21285691276', '21393999339', '13611653308', '17586234530']


In [2]:
res = get_html(NAVER_SHOPPING_BASE_URL, KUKU_AC_25W20FPMO)[2]['body']
print("Raw : ", res)
print("1 : ", res.splitlines())
print("2 : ", res.strip('\n\t\s'))
import re
print("3 : ", re.sub(r"[\n\t\s]*", "", res))
print("4 : ", res.replace("\n", "").replace("\t", "").replace(" ", ""))

IndexError: list index out of range

In [3]:
res = []
for product_id in product_arr:
    res.append(get_html(NAVER_SHOPPING_BASE_URL, product_id))
    print(len(res))

1
2
3
4
5
6
7


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options
options.headless = True
# Jupiter에서 driver 정상 동작 안함. 
# TypeError: to_capabilities() missing 1 required positional argument: 'self' 발생!
driver = webdriver.Chrome(executable_path="C:\\Users\\CheonYoungJo\\Desktop\\Local_Repo\\Algorithm\\chromedriver.exe", options=options)
driver.implicitly_wait(3)

#   if existReview:     리뷰가 존재하면 실행
#       page = parsing  페이지 몇개인지 찾기
#       while page:     모든 페이지 실행
#          get_tag
#          if existPage:   페이지가 더 있으면 스크립트 실행
#             driver.execution_script()
#             sleep()

def get_html2(_url, _id):
    r = requests.get(_url + _id)
    html = BeautifulSoup(r.content, 'html.parser')
    li_size = len(html.find_all("li", attrs={'class': 'thumb_nail'}))
    print(li_size)
    
get_html2("https://search.shopping.naver.com/detail/detail.nhn?nvMid=", "2131231231223213")

#container > div
#_review_paging > a.next_end


TypeError: to_capabilities() missing 1 required positional argument: 'self'

In [78]:
""" 
    Elasticsearch 적재
    Index Name Example : external-customer_review-제조사-모델명-200618
"""

from elasticsearch import Elasticsearch, helpers
import datetime, pprint, random, time
import pandas as pd

def make_index(_es, name):
    if es.indices.exists(index=name):
        es.indices.delete(index=name)
    print(es.indices.create(index=name))


es = Elasticsearch(["13.125.246.197:9200", "52.79.215.253:9200", "52.79.250.228:9200"])
index_name = "python_test3"
make_index(es, index_name)
print(es.info())
pp = pprint.PrettyPrinter(indent=4)

"""
    중복 저장 X 
    DateFrame use to changing the datetime type to pandas.timestamp type.
    Elasticsearch의 Bulk 함수를 이용하여 많은 데이터를 적재시키는 코드로 body array를 만들어 저장하는 방법.
"""

for office in res:
    body = [
        {
            "_index": index_name,
            "_source": x
        }
        for x in office
    ]
    res = helpers.bulk(es, body)
    pp.pprint(res)
    time.sleep(1)
    
es.indices.refresh(index=index_name)

# Test Code
# _id = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6', 'test7']
# _int = [1,2,3,4,5,6,7]
# _bool = [True, False, True, False, True, False, True]
# _date = [datetime.datetime.now(), datetime.datetime.now(), datetime.datetime.utcnow(), 
#          datetime.datetime.utcnow(), datetime.datetime.now(), datetime.datetime.utcnow(), datetime.datetime.now()]
# 
# df = pd.DataFrame([x for x in zip(_id, _int, _bool, _date)], columns=['ID', 'INT', 'BOOL', 'Dat'])
# print(df)
# for i in zip(df['ID'], df['INT'], df['BOOL'], df['Dat']):
#     print(i, type(i), i[3], type(i[3]))
#     a = i[3].to_pydatetime()
#     print(a, type(a))
# 
# body = [
#     {
#         "_index": "python_test2",
#         "_source": {
#             "Id": x[0],
#             "In": random.randint(0,10),
#             "Bol": x[2],
#             "Dat": datetime.datetime.utcnow()
#         }
#     }
#     # for x in zip(_id, _int, _bool, _date)
#     for x in zip(df['ID'], df['INT'], df['BOOL'], df['Dat'])
# ]
# 
# pp.pprint(body)
# 
# # res = es.bulk(index="python_test", body=body, doc_type='doc')
# res = helpers.bulk(es, body)
# pp.pprint(res)
# 
# es.indices.refresh(index="python_test2")


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python_test3'}
{'name': 'els-1', 'cluster_name': 'elk-bigdata', 'cluster_uuid': 'J9cMv8YXRkeq0ThyNLAYYQ', 'version': {'number': '7.6.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'aa751e09be0a5072e8570670309b1f12348f023b', 'build_date': '2020-02-29T00:15:25.529771Z', 'build_snapshot': False, 'lucene_version': '8.4.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
(20, [])
(20, [])
(20, [])
(20, [])
(20, [])
(20, [])
(20, [])


{'_shards': {'total': 2, 'successful': 2, 'failed': 0}}